In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np

# The Oracle 
def create_oracle(qc, dataset, guess_element, num_qubits):

    for i in range(len(dataset)):
        
        if dataset[i] > guess_element:
            binary_index = format(i, f'0{num_qubits}b')
            
            
            for qubit in range(num_qubits):
                if binary_index[qubit] == '0':
                    qc.x(qubit)
            
            qc.mcx(list(range(num_qubits - 1)), num_qubits - 1)
            
            for qubit in range(num_qubits):
                if binary_index[qubit] == '0':
                    qc.x(qubit)
    
    return qc

# Grover Diffusion Operator
def diffusion_operator(qc, num_qubits):

    # Hadamard gates
    qc.h(range(num_qubits))
    
    # X gates 
    qc.x(range(num_qubits))
    
    # Hadamard gate to the last qubit
    qc.h(num_qubits - 1)
    
    # Apply multi-controlled X gate (diffusion operator)
    qc.mcx(list(range(num_qubits - 1)), num_qubits - 1)
    
    # Apply Hadamard gate to the last qubit again
    qc.h(num_qubits - 1)
    # Reapply X gates to all qubits
    qc.x(range(num_qubits))
    # Apply Hadamard gates to all qubits again
    qc.h(range(num_qubits))
    
    return qc

# Grover's algorithm circuit
def grover_algorithm(dataset, guess_element):
    num_qubits = int(np.ceil(np.log2(len(dataset))))
    
    qc = QuantumCircuit(num_qubits, num_qubits)
    
    # create superposition
    qc.h(range(num_qubits))
    
    # Apply the Oracle
    qc = create_oracle(qc, dataset, guess_element, num_qubits)
    
    # Apply the Grover Diffusion Operator
    qc = diffusion_operator(qc, num_qubits)
    
    # Measure the qubits
    qc.measure(range(num_qubits), range(num_qubits))
    
    return qc


def execute_circuit(qc):
    backend = AerSimulator()
    compiled_qc = transpile(qc, backend)
    result = backend.run(compiled_qc, shots=1000).result()
    counts = result.get_counts(qc)
    return counts


def execute_grover(qc):
    counts = execute_circuit(qc)
    return counts


dataset = [32, 100, 1, 39 , 2, 0, 12]
n = len(dataset)
remaining_indices = list(range(n))

while remaining_indices:

    # Randomly select an index from the remaining indices
    random_index = np.random.choice(remaining_indices)
    guess_element = dataset[random_index]
    
    # Apply Grovers algorithm to find indices with elements greater than the guess_element
    qc = grover_algorithm(dataset, guess_element)
    
    counts = execute_grover(qc)
    
    
    
    new_remaining_indices = [int(index, 2) for index in counts if counts[index] > 0]
    
    # Filter out indices where the dataset value is not greater than the guess_element
    remaining_indices = [index for index in new_remaining_indices if index < len(dataset) and dataset[index] > guess_element]
    
    # If only one index remains, it's the maximum element
    if len(remaining_indices) == 1:
        max_index = remaining_indices[0]
        break
    
print("\n")

print(f"The index of the maximum element is: {max_index}")
print(f"The maximum element is: {dataset[max_index]}")

NameError: name 'max_index' is not defined